In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
## Load the dataset 
data=pd.read_csv("Churn_Modelling.csv")
data.head()

In [ ]:
## preprocess the data
## Drop irrelevant columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

In [ ]:
## Encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
## Onehot encode Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

In [ ]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

In [ ]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

In [ ]:
## Combine one hot encoder columns with the original data 
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

In [9]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [11]:
## Divide the dataset into the independent and dependent features
X=data.drop('Exited',axis=1)
y=data['Exited']

## Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [ ]:
X_train

In [13]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [ ]:
data

# ANN Implementation

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [ ]:
(X_train.shape[1],)

In [20]:
## Build Our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=((X_train.shape[1],))),  ## HL1 Connected with input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid') ## output layer
]

)

In [ ]:
model.summary()

In [ ]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [27]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [52]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [53]:
## Set up Early Stopping  
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [54]:
## Train the model 
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3297 - accuracy: 0.8649 - val_loss: 0.3546 - val_accuracy: 0.8535
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3301 - accuracy: 0.8631 - val_loss: 0.3471 - val_accuracy: 0.8590
Epoch 3/100
250/250 [==============================] - 3s 10ms/step - loss: 0.3290 - accuracy: 0.8650 - val_loss: 0.3445 - val_accuracy: 0.8610
Epoch 4/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3249 - accuracy: 0.8676 - val_loss: 0.3483 - val_accuracy: 0.8660
Epoch 5/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3216 - accuracy: 0.8661 - val_loss: 0.3529 - val_accuracy: 0.8540
Epoch 6/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3208 - accuracy: 0.8691 - val_loss: 0.3480 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 2s 10ms/step - loss: 0.3176 - accuracy: 0.8668 - val_loss: 0.3574 - val_accuracy: 0.

In [55]:
model.save('model.h5')

c:\Users\rohit\anaconda3\envs\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [58]:
## Load Tensorboard Extension 
%reload_ext tensorboard

In [59]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6013 (pid 984), started 0:01:00 ago. (Use '!kill 984' to kill it.)

In [ ]:
### Load the pickle file